In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam

from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau

In [43]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/jmnote/zdata/master/logistic-regression/special-sales.csv')
df

,date,weekday,busy_day,high_temperature,special_sales
0,2002-08-05,Mon,0,28,1
1,2002-08-06,Tue,0,24,0
2,2002-08-07,Wed,1,26,0
3,2002-08-08,Thu,0,24,0
4,2002-08-09,Fri,0,23,0
5,2002-08-10,Sat,1,28,1
6,2002-08-11,Sun,1,24,0
7,2002-08-12,Mon,0,26,1
8,2002-08-13,Tue,0,25,0
9,2002-08-14,Wed,1,28,1


In [44]:
Label = df['special_sales']
InputFeature = df[['busy_day', 'high_temperature']]

In [45]:
model = Sequential()
model.add(Dense(3, activation='linear', input_shape=(2, )))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

In [46]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 3)                 9         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 4         
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________


In [51]:
# Call-back 함수
# CheckPoint: Epoch 마다 validation 성능을 검증하여, best performance 일 경우 저장
CP = ModelCheckpoint(filepath='models/-{epoch:03d}-{loss:.4f}-{accuracy:.4f}.hdf5',
            monitor='loss', verbose=1, save_best_only=True, mode='min')

# Learning Rate 줄여나가기
LR = ReduceLROnPlateau(monitor='loss',factor=0.8,patience=3, verbose=1, min_lr=1e-8)

CALLBACK = [CP, LR]

In [52]:
model.fit(x=InputFeature, y=Label, epochs=100, shuffle=True, batch_size=3, callbacks=CALLBACK)

Epoch 1/100
7/7 [==============================] - 0s 3ms/step - loss: 0.5512 - accuracy: 0.7619

Epoch 00001: loss improved from inf to 0.55123, saving model to models\-001-0.5512-0.7619.hdf5
Epoch 2/100
7/7 [==============================] - 0s 3ms/step - loss: 0.5517 - accuracy: 0.7619

Epoch 00002: loss did not improve from 0.55123
Epoch 3/100
7/7 [==============================] - 0s 20ms/step - loss: 0.5513 - accuracy: 0.7619

Epoch 00003: loss did not improve from 0.55123
Epoch 4/100
7/7 [==============================] - 0s 2ms/step - loss: 0.5511 - accuracy: 0.7619

Epoch 00004: loss improved from 0.55123 to 0.55115, saving model to models\-004-0.5511-0.7619.hdf5

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0001801439793780446.
Epoch 5/100
7/7 [==============================] - 0s 2ms/step - loss: 0.5512 - accuracy: 0.7619

Epoch 00005: loss did not improve from 0.55115
Epoch 6/100
7/7 [==============================] - 0s 1ms/step - loss: 0.5513 - accuracy: 0.7

In [58]:
model.load_weights("./models/-100-0.5505-0.7619.hdf5")

In [8]:
from sklearn.metrics import roc_auc_score

print(roc_auc_score(Label, model.predict(InputFeature)))

0.8798076923076923


In [16]:
from flask import Flask
from flask import render_template

In [34]:
app = Flask(__name__)

In [35]:
@app.route('/sale/<busy_day>/<high_temperature>')
def get_auc(busy_day=None, high_temperature=None):
    data = pd.DataFrame({
        'busy_day': [int(busy_day)],
        'high_temperature':[float(high_temperature)]
    })
    return "auc: " + str(model.predict(data)[0][0])

In [36]:
app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.55.187:5000 (Press CTRL+C to quit)
192.168.55.187 - - [06/Jul/2022 17:28:47] "GET /sale/1/30 HTTP/1.1" 200 -
192.168.55.187 - - [06/Jul/2022 17:28:47] "GET /sale/1/30 HTTP/1.1" 200 -
[2022-07-06 17:28:51,697] ERROR in app: Exception on /sale/1/2- [GET]
Traceback (most recent call last):
  File "c:\Users\gmlkd\miniforge3\envs\fastcampus\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\gmlkd\miniforge3\envs\fastcampus\lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\gmlkd\miniforge3\envs\fastcampus\lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\gmlkd\miniforge3\envs\fastcampus\lib\site-packages\flask\app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.vi